In [1]:
# flake8: noqa: E402, B018, F401
%load_ext autoreload
%autoreload 2

In [2]:
import cProfile
import pstats

from icecream import ic

from classes import Regular, Slotted, Weak, attr_reader, attr_writer, bench, invoker

N_RUNS = 10
N_LOOPS = 100_000

In [7]:
invoker(Weak)

Filename: /home/ari/workdir/deep_python_23b_aalitvinov/08/classes.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    33     62.5 MiB     62.5 MiB           1   @profile
    34                                         def invoker(cls: type) -> Any:
    35     62.5 MiB      0.0 MiB           1       return cls({rng.random()}, {rng.random()})




In [44]:
bench(lambda: invoker(Regular), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: invoker(Slotted), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: invoker(Weak), n_runs=N_RUNS, n_loops=N_LOOPS)

1811 ns ± 819.9 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 1081 ns
1298 ns ± 266.9 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 949 ns
1783 ns ± 266.0 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 1506 ns


{'mean': 1783, 'stdev': 266.0, 'min': 1506}

In [45]:
bench(lambda: attr_reader(Regular), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: attr_reader(Slotted), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: attr_reader(Weak), n_runs=N_RUNS, n_loops=N_LOOPS)

1805 ns ± 630.2 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 1250 ns
2040 ns ± 421.4 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 1329 ns
2483 ns ± 739.1 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 1688 ns


{'mean': 2483, 'stdev': 739.1, 'min': 1688}

In [46]:
bench(lambda: attr_writer(Regular), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: attr_writer(Slotted), n_runs=N_RUNS, n_loops=N_LOOPS)
bench(lambda: attr_writer(Weak), n_runs=N_RUNS, n_loops=N_LOOPS)

3868 ns ± 507.7 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 3145 ns
3392 ns ± 425.7 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 2544 ns
3441 ns ± 1413.9 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
best time 1956 ns


{'mean': 3441, 'stdev': 1413.9, 'min': 1956}

In [64]:
from time import perf_counter, perf_counter_ns
import profile

pr = profile.Profile(perf_counter_ns)
pr.run("invoker(Regular)")
pr.print_stats()

         8 function calls in 279283.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1 56008.000 56008.000 80707.000 80707.000 :0(exec)
        2 2059.000 1029.500 2059.000 1029.500 :0(random)
        1 92625.000 92625.000 92625.000 92625.000 :0(setprofile)
        1 5958.000 5958.000 24699.000 24699.000 <string>:1(<module>)
        1 3142.000 3142.000 3142.000 3142.000 classes.py:12(__init__)
        1 13540.000 13540.000 18741.000 18741.000 classes.py:31(invoker)
        1 105951.000 105951.000 279283.000 279283.000 profile:0(invoker(Regular))
        0    0.000             0.000          profile:0(profiler)




In [62]:
pr.print_stats()

         9 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 cProfile.py:118(__exit__)
        1    0.000    0.000    0.000    0.000 classes.py:12(__init__)
        1    0.000    0.000    0.000    0.000 classes.py:31(invoker)
        1    0.000    0.000    0.000    0.000 classes.py:41(attr_writer)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        4    0.000    0.000    0.000    0.000 {method 'random' of '_random.Random' objects}


